In [ ]:
# are the future functions actually necessary?
from __future__ import absolute_import, division, print_function, unicode_literals
import imp

from globalConstants import Const
from datasetData import DatasetData
from pipeline import Pipeline
from autoEncoder import AutoEncoder 
from plotter import Plotter
from importanceData import ImportanceData
from stepwiseData import StepwiseData

from data_read import *
from helperFunctions import *
from plotHelperFunctions import *
from losses import *

import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib as mpl

import random
import time
from collections import Counter
import sys

np.set_printoptions(precision=3, suppress=True)
# allows for automatic reloading of imports and makes it unncessecary to restart the kernel
# whenever a function is changed
%load_ext autoreload
%autoreload 2

In [ ]:
c = Const()

In [ ]:
trainData, valData, testData = DatasetData\
    .initialize_train_val_test_datasets(
        *make_train_val_test_from_TIS_and_TPS(c))

In [ ]:
list_var_names = ["$MCG$", "$N_{w,4}$", "$N_{w,3}$", "$N_{w,2}$", "$N_{sw,3-4}$",
    "$N_{sw,2-3}$", "$F4$", "$R_g$", "$5^{12}6^{2}$", "$5^{12}$",
    "$CR$", "$N_{s,2}$", "$N_{s,3}$", "$N_{c,2}$", "$N_{c,3}$",
    "$N_{s,4}$", "$N_{c,4}$", "$5^{12}6^{3}$", "$5^{12}6^{4}$", 
    "$4^{1}5^{10}6^{2}$", "$4^{1}5^{10}6^{3}$", "$4^{1}5^{10}6^{4}$"]

reduced_list_var_names = [
    "MCG", "5^{12}6^{2}", "5^{12}",
    "CR", "R_g", "F4", "N_{w,3}"
    ]

reduced_name_to_list_position = {reduced_list_var_names[i]: i for i in range(len(reduced_list_var_names))}


In [ ]:
#print_coverage(list_var_names, trainData)


In [ ]:
c = Const()
pipeline = Pipeline(c, reduced_list_var_names, trainData.snapshots)
print(get_size(pipeline))

In [ ]:
train_grid_snapshots, \
    train_norm_snapshots, \
    train_pB_dict, \
    train_pBs, \
    train_pB_weights = pipeline.rbnga(trainData)
train_trimmed_grid_snapshots, \
    train_trimmed_norm_snapshots, \
    train_trimmed_labels, \
    train_trimmed_weights, \
    train_trimmed_pB_dict, \
    train_trimmed_pBs, \
    train_trimmed_pB_weights, \
    train_trimmed_balanced_pB_weights, \
    train_trimmed_balanced_hc_weights = pipeline.rbngatnb(trainData)

val_grid_snapshots, \
    val_norm_snapshots, \
    val_pB_dict, \
    val_pBs, \
    val_pB_weights = pipeline.rbnga(valData)
val_trimmed_grid_snapshots, \
    val_trimmed_norm_snapshots, \
    val_trimmed_labels, \
    val_trimmed_weights, \
    val_trimmed_pB_dict, \
    val_trimmed_pBs, \
    val_trimmed_pB_weights, \
    val_trimmed_balanced_pB_weights, \
    val_trimmed_balanced_hc_weights = pipeline.rbngatnb(valData)

train_trimmed_pBl_pBbw_ds = tf.data.Dataset.from_tensor_slices(
    ({c.input_name: train_trimmed_norm_snapshots},
    {c.output_name_1: train_trimmed_pBs,
    c.output_name_2: train_trimmed_norm_snapshots},
    {c.output_name_1: train_trimmed_balanced_pB_weights,
    c.output_name_2: train_trimmed_balanced_hc_weights})) \
        .shuffle(pipeline.snapshot_cnt) \
        .batch(c.batch_size)
val_trimmed_pBl_pBbw_ds = tf.data.Dataset.from_tensor_slices(
    ({c.input_name: val_trimmed_norm_snapshots},
    {c.output_name_1: val_trimmed_pBs, 
    c.output_name_2: val_trimmed_norm_snapshots},
    {c.output_name_1: val_trimmed_balanced_pB_weights, 
    c.output_name_2: val_trimmed_balanced_hc_weights})) \
        .shuffle(pipeline.snapshot_cnt) \
        .batch(c.batch_size)

In [ ]:
all_1D_mean_representations = get_all_1D_correlated_representations(
    train_trimmed_grid_snapshots, get_means_from_tuples)

all_1D_mode_representations = get_all_1D_correlated_representations(
    train_trimmed_grid_snapshots, get_modes_from_tuples)

#all_2D_mean_representations = get_all_2D_correlated_representations(
#    train_grid_snapshots, get_means_from_tuples)

#all_2D_mode_representations = get_all_2D_correlated_representations(
#    train_grid_snapshots, get_modes_from_tuples)

all_2D_trimmed_mean_representations = get_all_2D_correlated_representations(
    train_trimmed_grid_snapshots, get_means_from_tuples)

all_2D_trimmed_mode_representations = get_all_2D_correlated_representations(
    train_trimmed_grid_snapshots, get_modes_from_tuples)


In [ ]:
autoencoder, autoencoder_1, autoencoder_2, \
    encoder, decoder_1, decoder_2 = \
    AutoEncoder.model(len(reduced_list_var_names), differenceOfLogs, c)
history = autoencoder.fit(
    x=train_trimmed_pBl_pBbw_ds,
    epochs=c.epochs,
    validation_data=val_trimmed_pBl_pBbw_ds,
    callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=3)])

In [ ]:
formula_components = get_encoder_formula(encoder, reduced_list_var_names)
for component in formula_components:
    print(component)

In [ ]:
paths, labels, _, origins = make_paths_from_TPS_data(1, c)
for i, path in enumerate(paths):
    map_path_on_2D_latent_space(
        pipeline=pipeline,
        path=path,
        encoder=encoder,
        skip=10,
        pre_stamp=str(i),
        const=c)

In [ ]:
for i, path in enumerate(paths):
    map_path_on_1D_latent_space(
        pipeline=pipeline,
        path=path,
        encoder=encoder,
        skip=10,
        pre_stamp=str(i),
        const=c)

In [ ]:
for i, path in enumerate(paths):
    map_path_on_timed_1D_latent_space(
        pipeline=pipeline,
        path=path,
        encoder=encoder,
        skip=10,
        pre_stamp=str(i),
        const=c)

In [ ]:
Plotter.plot_super_map(
    used_variable_names=reduced_list_var_names,
    name_to_list_position=reduced_name_to_list_position,
    lower_bound=pipeline.lower_bound,
    upper_bound=pipeline.upper_bound,
    const=c,
    pre_stamp="CorrelatedMean",
    method=Plotter.calc_represented_map_generated,
    model=autoencoder_1,
    minima=np.amin(train_trimmed_norm_snapshots, axis=0),
    maxima=np.amax(train_trimmed_norm_snapshots, axis=0),
    representations=all_2D_trimmed_mean_representations)
pass

Plotter.plot_super_map(
    used_variable_names=reduced_list_var_names,
    name_to_list_position=reduced_name_to_list_position,
    lower_bound=pipeline.lower_bound,
    upper_bound=pipeline.upper_bound,
    const=c,
    pre_stamp="CorrelatedMode",
    method=Plotter.calc_represented_map_generated,
    model=autoencoder_1,
    minima=np.amin(train_trimmed_norm_snapshots, axis=0),
    maxima=np.amax(train_trimmed_norm_snapshots, axis=0),
    representations=all_2D_trimmed_mode_representations)
pass

In [ ]:
Plotter.plot_super_scatter(
    used_variable_names=reduced_list_var_names,
    name_to_list_position=reduced_name_to_list_position,
    lower_bound=pipeline.lower_bound,
    upper_bound=pipeline.upper_bound,
    const=c,
    pre_stamp="CorrelatedMean",
    model=autoencoder_2,
    minima=np.amin(train_trimmed_norm_snapshots, axis=0),
    maxima=np.amax(train_trimmed_norm_snapshots, axis=0),
    method=Plotter.calc_represented_scatter_generated,
    representations=all_1D_mean_representations,
    max_row_len=6)

Plotter.plot_super_scatter(
    used_variable_names=reduced_list_var_names,
    name_to_list_position=reduced_name_to_list_position,
    lower_bound=pipeline.lower_bound,
    upper_bound=pipeline.upper_bound,
    const=c,
    pre_stamp="CorrelatedMode",
    model=autoencoder_2,
    minima=np.amin(train_trimmed_norm_snapshots, axis=0),
    maxima=np.amax(train_trimmed_norm_snapshots, axis=0),
    method=Plotter.calc_represented_scatter_generated,
    representations=all_1D_mode_representations,
    max_row_len=6)

In [ ]:
plot_ground_truth(
    reduced_list_var_names=reduced_list_var_names,
    reduced_name_to_list_position=reduced_name_to_list_position,
    pipeline=pipeline,
    const=c,
    grid_snapshots=train_grid_snapshots,
    labels=trainData.labels,
    weights=trainData.weights,
    pre_stamp="GroundTruthTrimmed_Train")

plot_ground_truth(
    reduced_list_var_names=reduced_list_var_names,
    reduced_name_to_list_position=reduced_name_to_list_position,
    pipeline=pipeline,
    const=c,
    grid_snapshots=train_trimmed_grid_snapshots,
    labels=train_trimmed_labels,
    weights=train_trimmed_weights,
    pre_stamp="GroundTruth_Train")

In [ ]:
methods_used = [
    logBinaryNegLikelihood, logBinomialNegLikelihood,
    differenceOfLogs, logLoss]

In [ ]:
for method in methods_used:
    print(str(method).split(" ")[1]+"\n")
    super_pBsLabels_pBbWeights = plot_with_different_settings(
        reduced_list_var_names=reduced_list_var_names,
        reduced_name_to_list_position=reduced_name_to_list_position,
        const=c,
        train_ds=train_trimmed_pBl_pBbw_ds,
        val_ds=val_trimmed_pBl_pBbw_ds,
        loss_function=method,
        pipeline=pipeline,
        pre_stamp="trimmed_pBsLabels_pBb+hcbWeights_{}"
            .format(function_to_str(method)),
        minima=np.amin(train_trimmed_norm_snapshots, axis=0),
        maxima=np.amax(train_trimmed_norm_snapshots, axis=0))

In [ ]:
train_grid_snapshots, \
    train_norm_snapshots, \
    train_labels, \
    train_weights, \
    train_s_pB_dict, \
    train_s_pBs, \
    train_pB_weights, \
    train_s_balanced_pB_weights, \
    train_balanced_hc_weights = pipeline.rbngasb(trainData)

val_grid_snapshots, \
    val_norm_snapshots, \
    val_labels, \
    val_weights, \
    val_s_pB_dict, \
    val_s_pBs, \
    val_pB_weights, \
    val_s_balanced_pB_weights, \
    val_balanced_hc_weights = pipeline.rbngasb(valData)

train_squeezed_pBl_pBbw_ds = tf.data.Dataset.from_tensor_slices(
    ({c.input_name: train_norm_snapshots}, 
    {c.output_name_1: train_s_pBs, 
    c.output_name_2: train_norm_snapshots},
    {c.output_name_1: train_s_balanced_pB_weights, 
    c.output_name_2: train_balanced_hc_weights})) \
        .shuffle(pipeline.snapshot_cnt) \
        .batch(c.batch_size)
val_squeezed_pBl_pBbw_ds = tf.data.Dataset.from_tensor_slices(
    ({c.input_name: val_norm_snapshots}, 
    {c.output_name_1: val_s_pBs, 
    c.output_name_2: val_norm_snapshots},
    {c.output_name_1: val_s_balanced_pB_weights, 
    c.output_name_2: val_balanced_hc_weights})) \
        .shuffle(pipeline.snapshot_cnt) \
        .batch(c.batch_size)

In [ ]:
for method in methods_used:
    print(str(method).split(" ")[1]+"\n")
    super_pBsLabels_pBbWeights = plot_with_different_settings(
        reduced_list_var_names=reduced_list_var_names,
        reduced_name_to_list_position=reduced_name_to_list_position,
        const=c,
        train_ds=train_squeezed_pBl_pBbw_ds,
        val_ds=val_squeezed_pBl_pBbw_ds,
        loss_function=method,
        pipeline=pipeline,
        pre_stamp="squeezed_pBsLabels_pBb+hcbWeights_{}"\
            .format(function_to_str(method)),
        minima=np.amin(train_norm_snapshots, axis=0),
        maxima=np.amax(train_norm_snapshots, axis=0))

In [ ]:
plot_encoder_decoder(
    const=c,
    loss_function=differenceOfLogs,
    reduced_list_var_names=reduced_list_var_names,
    reduced_name_to_list_position=reduced_name_to_list_position,
    train_ds=train_trimmed_pBl_pBbw_ds,
    val_ds=val_trimmed_pBl_pBbw_ds,
    pipeline=pipeline)
pass

In [ ]:
shooting_points, shooting_labels = read_shooting_points(
    "total_data_till_982mc_280K.txt")

shootingData = DatasetData(
    shooting_points,
    shooting_labels,
    np.ones(len(shooting_labels)),
    flag="Shooting")
shooting_grid_snapshots, \
    shooting_norm_snapshots, \
    shooting_pB_dict, \
    shooting_pBs, \
    shooting_pB_weights = pipeline.rbnga(shootingData)
shooting_trimmed_grid_snapshots, \
    shooting_trimmed_norm_snapshots, \
    shooting_trimmed_labels, \
    shooting_trimmed_weights, \
    shooting_trimmed_pB_dict, \
    shooting_trimmed_pBs, \
    shooting_trimmed_pB_weights, \
    shooting_trimmed_balanced_pB_weights, \
    shooting_trimmed_balanced_hc_weights = pipeline.rbngatnb(shootingData)

In [ ]:
def MCG_BigCage(x):
    return 68.14 - 0.4286*x

def now_BigCage(x):
    return 30 - 0.0553*x

def make_linspaces(function, pipeline, x_int, y_int):
    xs = np.linspace(
        pipeline.lower_bound[x_int],
        pipeline.upper_bound[x_int],
        11)
    ys = np.array([function(x) for x in xs])
    xs = (xs - pipeline.lower_bound[x_int]) / (pipeline.upper_bound[x_int] - pipeline.lower_bound[x_int])
    ys = (ys - pipeline.lower_bound[y_int]) / (pipeline.upper_bound[y_int] - pipeline.lower_bound[y_int])
    return np.array(xs), np.array(ys)
c = Const()

def plot_one_map(function, x_int, y_int, stamp, method, **kwargs):
    fig, ax = plt.subplots(1,1)
    plt.plot(
        *make_linspaces(function, pipeline, x_int, y_int),
        c="r")
    plt.imshow(
        np.maximum(
            np.transpose(
                method(
                    x_pos=x_int,
                    y_pos=y_int,
                    resolution=c.resolution,
                    **kwargs)[0])[::-1],
            c.logvmin / 2),
        cmap=c.cmap,
        interpolation='nearest',
        norm=mpl.colors.LogNorm(
            vmin=c.logvmin,
            vmax=1.0-c.logvmin),
        extent=[0, 1, 0, 1])
    ax.set_xticks(np.linspace(0,1,3))
    ax.set_xticklabels(
        np.around(
            np.linspace(
                pipeline.lower_bound[x_int], 
                pipeline.upper_bound[x_int],
                3),
            2),
        rotation=60)
    ax.set_yticks(np.linspace(0,1,3))
    ax.set_yticklabels(np.around(
        np.linspace(
            pipeline.lower_bound[y_int], 
            pipeline.upper_bound[y_int],
            3),
        2))
    ax.set_xlabel(
        "${}$".format(reduced_list_var_names[x_int]),
        fontsize=c.subfig_size * 10)
    ax.set_ylabel(
        "${}$".format(reduced_list_var_names[y_int]),
        fontsize=c.subfig_size * 10)
    plt.colorbar(extend="both")
    plt.tight_layout()
    plt.savefig("results/{}_x{}_y_{}.png".format(stamp, x_int, y_int))
    plt.show()

"""
plot_one_map(
    MCG_BigCage,
    0,
    1,
    method=Plotter.calc_map_given,
    grid_snapshots=shooting_grid_snapshots, 
    labels=shootingData.labels, 
    weights=shootingData.weights,
    stamp="DividingSurfaceTest_0.7shootingData_MCG_BigCage" + c.data_stamp)
"""
plot_one_map(
    MCG_BigCage,
    0,
    1,
    method=Plotter.calc_map_given,
    grid_snapshots=train_grid_snapshots, 
    labels=trainData.labels, 
    weights=trainData.weights,
    stamp="DividingSurfaceTest_0.7trainData_MCG_BigCage" + c.data_stamp)
"""
plot_one_map(
    now_BigCage,
    6,
    1,
    method=Plotter.calc_map_given,
    grid_snapshots=shooting_grid_snapshots, 
    labels=shootingData.labels, 
    weights=shootingData.weights,
    stamp="DividingSurfaceTest_0.7shootingData_now3_BigCage" + c.data_stamp)
"""
plot_one_map(
    now_BigCage,
    6,
    1,
    method=Plotter.calc_map_given,
    grid_snapshots=train_grid_snapshots, 
    labels=trainData.labels, 
    weights=trainData.weights,
    stamp="DividingSurfaceTest_0.7trainData_now3_BigCage" + c.data_stamp)

c.epochs = 10
autoencoder, autoencoder_1, autoencoder_2, \
    encoder, decoder_1, decoder_2 = \
    AutoEncoder.model(len(reduced_list_var_names), differenceOfLogs, c)
#autoencoder.fit(train_ds_batch,epochs=EPOCHS, class_weight=class_weight)
history = autoencoder.fit(
    x=train_trimmed_pBl_pBbw_ds,
    epochs=c.epochs,
    validation_data=val_trimmed_pBl_pBbw_ds,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)])

plot_one_map(
    MCG_BigCage,
    0,
    1,
    method=Plotter.calc_map_generated,
    model=autoencoder_1, 
    minima=np.amin(train_trimmed_norm_snapshots, axis=0),
    maxima=np.amax(train_trimmed_norm_snapshots, axis=0),
    stamp="DividingSurfaceTest_Gen_MCG_BigCage" + c.data_stamp)
plot_one_map(
    now_BigCage,
    6,
    1,
    method=Plotter.calc_map_generated,
    model=autoencoder_1, 
    minima=np.amin(train_trimmed_norm_snapshots, axis=0),
    maxima=np.amax(train_trimmed_norm_snapshots, axis=0),
    stamp="DividingSurfaceTest_Gen_now3_BigCage" + c.data_stamp)
pass

In [ ]:
def get_percentage_of_range_retained(outlier_cutoff):
    snapshots = trainData.snapshots
    span = np.amax(snapshots, axis=0) - np.amin(snapshots, axis=0)
    percentile_span = np.percentile(snapshots, 100 - outlier_cutoff, axis=0) \
        - np.percentile(snapshots, outlier_cutoff, axis=0)
    print(np.mean(percentile_span/span))
    
def estimate_reduction_on_AA_and_AB():
    reducer = Reducer(
        reduced_list_var_names,
        c.name_to_list_position)
    reduced_snapshots = reducer.reduce_snapshots(trainData.snapshots)
    bounder = Bounder(reduced_snapshots, c.outlier_cutoff)
    bound_snapshots = bounder.bound_snapshots(reduced_snapshots)

    all_AA_frames = len([1 for i, label in enumerate(trainData.labels) if label == 0])
    all_AB_frames = len([1 for i, label in enumerate(trainData.labels) if label == 1])
    bound_AA_frames = len([1 for i, label in enumerate(trainData.labels) if (label == 0 and bound_snapshots[i][0] == bounder.upper_bound[0])])
    bound_AB_frames = len([1 for i, label in enumerate(trainData.labels) if (label == 1 and bound_snapshots[i][0] == bounder.upper_bound[0])])

    print(bound_AA_frames/all_AA_frames)
    print(bound_AB_frames/all_AB_frames)

In [ ]:
def plot_one_map_scatter(dataset, x, y, function):
        xs, ys = zip(*[(snapshot[x],snapshot[y]) for snapshot in dataset.snapshots])
        xlin = np.linspace(min(xs), max(xs),11)
        ylin = np.array([function(x) for x in xlin])
        weights = dataset.weights*(1/max(dataset.weights))
        labels = [[label, 0.0, 1 - label, 0.5] for i,label in enumerate(dataset.labels)]
        weights = [[0, weights[i], weights[i], 0.5] for i,_ in enumerate(labels)]
        plt.plot(xlin, ylin, c="k")
        plt.scatter(xs, ys, s=0.5, c=labels)
        plt.xlim((min(xs),max(xs)))
        plt.ylim((min(ys),max(ys)))
        plt.savefig("results/scatter_{}_{}_labels".format(dataset.flag, function_to_str(function),))
        plt.show()
        plt.plot(xlin, ylin, c="k")
        plt.scatter(xs, ys, s=0.5, c=weights)
        plt.xlim((min(xs),max(xs)))
        plt.ylim((min(ys),max(ys)))
        plt.savefig("results/scatter_{}_{}_weights".format(dataset.flag, function_to_str(function),))
        plt.show()
        
plot_one_map_scatter(
    trainData,
    0,
    8,
    MCG_BigCage)
plot_one_map_scatter(
    shootingData,
    0,
    8,
    MCG_BigCage)

In [ ]:
train_trimmed_pBsl_ds = tf.data.Dataset.from_tensor_slices(
    ({c.input_name: train_trimmed_norm_snapshots}, 
    {c.output_name_1: train_trimmed_pBs, 
    c.output_name_2: train_trimmed_norm_snapshots})) \
        .shuffle(pipeline.snapshot_cnt) \
        .batch(c.batch_size)
val_trimmed_pBsl_ds = tf.data.Dataset.from_tensor_slices(
    ({c.input_name: val_trimmed_norm_snapshots}, 
    {c.output_name_1: val_trimmed_pBs, 
    c.output_name_2: val_trimmed_norm_snapshots})) \
        .shuffle(pipeline.snapshot_cnt) \
        .batch(c.batch_size)
for method in methods_used:
    super_pBsLabels = plot_with_different_settings(
        train_trimmed_pBsl_ds, 
        val_trimmed_pBsl_ds, 
        method, 
        "trimmed_pBsLabels")

In [ ]:
stepO = StepwiseData(*dataO.stepwise_data())

In [ ]:
print(c.names_in_order)

In [ ]:
used, loss = stepO.bottom_up(
    used = [], 
    #unused = c.names_in_order, 
    unused = reduced_list_var_names,
    name_to_list_position = reduced_name_to_list_position,
    param_limit = 8, 
    epochs = 2, 
    repetitions = 1, 
    const = c)

In [ ]:
print(used, loss)

In [ ]:
used, loss = stepO.top_down( 
    #used = c.names_in_order, 
    used = reduced_list_var_names, 
    unused = [], 
    name_to_list_position = reduced_name_to_list_position,
    param_limit = 3, 
    epochs = 2, 
    repetitions = 1, 
    const = c)

In [ ]:
print(used, loss)

In [ ]:
impO = ImportanceData(
    *dataO.importance_data(),
    c.corr_thresholds)

impO.measure_correlation()
pass

In [ ]:
val_ds = tf.data.Dataset.from_tensor_slices(
    ({c.input_name: dataO.val_norm_snapshots}, 
    {c.output_name_1: dataO.val_labels, 
    c.output_name_2: dataO.val_norm_snapshots}, 
    {c.output_name_1: dataO.val_weights, 
    c.output_name_2: dataO.val_weights})) \
        .shuffle(pipeline.snapshot_cnt) \
        .batch(c.batch_size)
print("val_ds generated")

modes = [["Mean", None],["HIPR", [-0.9,0.9]],["Shuffle", None]]
#modes = [["Perturb", 0.5]]

impO.plot_super_importance(
    subfig_size = c.subfig_size, i_s = var_order, 
    stamp = c.stamp, var_names = var_names, 
    repetitions = 1, modes = modes,
    loss_names = c.loss_names,
    val_ds = val_ds, model = autoencoder)

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices(
    ({c.input_name: dataO.test_norm_snapshots}, 
    {c.output_name_1: dataO.test_labels, 
    c.output_name_2: dataO.test_norm_snapshots}, 
    {c.output_name_1: dataO.test_weights, 
    c.output_name_2: dataO.test_weights})) \
        .shuffle(pipeline.snapshot_cnt) \
        .batch(c.batch_size)
print("test_ds generated")

In [ ]:
impO.reduced_set_importance([1], [2,3,4,5,6], "a", 1)

In [ ]:
model = autoencoder
print(model.name)
print(model.input_names)
print(model.output_names)

In [ ]:
train_grid_columns = np.transpose(train_grid_snapshots)
def rec_cols(used, unused, lim, p):
    if len(used) == lim:
        p.append(used)
        return
    else:
        for i in range(len(unused)):
            rec_cols(used+[unused[i]], unused[i+1:], lim, p)

def mean_pB_attributes(dimensions, train_grid_columns, dataO, tries):
    p = []
    rec_cols([],list(range(22)),dimensions, p)
    pB_uniques = []
    pB_unique_means = []
    pB_means = []
    pB_unique_zeroes = []
    for i in range(tries):
        choice = random.choice(p)
        short_grid_snapshots = []
        for j in choice:
            short_grid_snapshots.append(train_grid_columns[j])

        pB_dict, pBs = gridO.approximate_pB(np.transpose(short_grid_snapshots), dataO.train_labels, dataO.train_weights)
        pB_uniques.append(len(pB_dict)/len(pBs))
        pB_unique_means.append(np.mean([label for key, label in pB_dict.items()]))
        pB_means.append(np.mean(pBs))
        pB_unique_zeroes.append(len([label for key, label in pB_dict.items() if label == 0])/len(pBs))
    return np.mean(pB_uniques), np.mean(pB_unique_means), np.mean(pB_means), np.mean(pB_unique_zeroes)
over_list = []
for i in range(1,23):
    print(i)
    over_list.append(list(mean_pB_attributes(i, train_grid_columns, dataO, 5)))

print(over_list)

In [ ]:
over_list = [[1.4504122192394765e-05, 0.4575422359289151, 0.35037916491845283, 0.0], 
             [8.823341000373481e-05, 0.31197805032254855, 0.24989161958688205, 1.4987592932141258e-05], 
             [0.0011390570628427355, 0.254336987822713, 0.21065742919045424, 0.00041312574711337754], 
             [0.0031104090041590574, 0.18133452313340712, 0.2170178744045542, 0.0019471784043289969], 
             [0.019666622751407806, 0.14815356079849162, 0.1799223783174298, 0.016739932628352418], 
             [0.03723860852286394, 0.0766339470321534, 0.15529480428303702, 0.03335392112900087], 
             [0.10585688540897394, 0.04434703300207507, 0.12188629749425983, 0.10030736652279384], 
             [0.14584233293970758, 0.03858060089555661, 0.12155443609723238, 0.13996139486143125], 
             [0.2389070660457291, 0.030955704907381622, 0.08891275517831804, 0.2316479946238054], 
             [0.3461106592002669, 0.02109568034865818, 0.07299244764109925, 0.33821316466650786], 
             [0.40314715278037977, 0.023028707168210764, 0.06465460885978427, 0.3932804819236334], 
             [0.5168345720256579, 0.020024200833715332, 0.055430137658731636, 0.5062564739753744], 
             [0.6097532969682758, 0.01770556773880911, 0.04620075248938048, 0.5986738397608754], 
             [0.649903849756633, 0.019642215099454723, 0.040862941059977745, 0.6368426459870116], 
             [0.648240710411905, 0.018966941544334908, 0.04145155090339449, 0.6356276840180286], 
             [0.694312812820677, 0.019123793821338247, 0.037413562972544265, 0.6808039151460504], 
             [0.7417976167309884, 0.0178504232145612, 0.03518109760382279, 0.7283716342882285], 
             [0.7338169652299931, 0.018954433483644333, 0.035358386024052696, 0.7197073551612314], 
             [0.7123308003495493, 0.01780718726971691, 0.03705239598119732, 0.6994197142446192], 
             [0.7675257538819679, 0.01880599290810691, 0.03314824301337975, 0.7529500780200907], 
             [0.7685473275550522, 0.02063167015576706, 0.032021440958897376, 0.752516404766538], 
             [0.7835114721563158, 0.021141123648464055, 0.031053932448914478, 0.766812392805472]
            ]

In [ ]:
over_columns = np.transpose(over_list)
plt.scatter(list(range(1,23)),over_columns[1])
plt.ylim(-0.1,1.1)
plt.xlim(0,23)
plt.xlabel("Dimensions")
plt.ylabel("Mean")
plt.title("Mean pB value of all unique entries")
plt.show()
plt.close()
            
over_columns = np.transpose(over_list)
plt.scatter(list(range(1,23)),over_columns[2])
plt.ylim(-0.1,1.1)
plt.xlim(0,23)
plt.xlabel("Dimensions")
plt.ylabel("Mean")
plt.title("Mean pB value of all entries")
plt.show()
plt.close()
            
plt.figure()
plt.scatter(list(range(1,23)),over_columns[0], label = "Unique entries")
plt.scatter(list(range(1,23)),over_columns[3], label = "Unique entries = 0")
plt.xlabel("Dimensions")
plt.ylabel("Fraction")
plt.ylim(-0.1,1)
plt.xlim(0,23)
plt.legend(loc = "lower right")
plt.title("Fraction of unique entries of all entries")
plt.show()
plt.figure()

plt.scatter(list(range(1,23)),(over_columns[0]-over_columns[3])/over_columns[0])
plt.ylim(-0.1,1.1)
plt.xlim(0,23)
plt.xlabel("Dimensions")
plt.ylabel("Fraction")
plt.title("Fraction of non-zero entries of all unique entries")
plt.show()
plt.close()

In [ ]:
gridO.plot_distribution(train_grid_snapshots,6,20,var_names,"untrimmed")
gridO.plot_distribution(trimmed_keys,6,20,var_names,"trimmed_both")
gridO.plot_distribution(trimmed_back_keys,6,20,var_names,"trimmed_back")

In [ ]:
#print(len(pB_dict))
#print(max([label for key, label in pB_dict.items()]))
#print(min([label for key, label in pB_dict.items()]))
#print(len([label for key, label in pB_dict.items() if label > 0.0]))
#print(len([label for key, label in pB_dict.items() if label == 0.0]))
#print(len([label for key, label in pB_dict.items() if label < 0.25]))
#print(len([label for key, label in pB_dict.items() if label > 0.25]))

def broken_hist(xs, bins, y_lower_1, y_upper_1, y_lower_2, y_upper_2, filename):
    f, (ax, ax2) = plt.subplots(2, 1, sharex=True)
    # ax.hist([label for key, label in pB_dict.items()], 100)
    # ax2.hist([label for key, label in pB_dict.items()], 100)
    ax.hist(xs, bins)
    ax2.hist(xs, bins)
    ax.set_ylim(y_lower_2, y_upper_2)  # outliers only
    ax2.set_ylim(y_lower_1, y_upper_1)  # most of the data
    # hide the spines between ax and ax2
    ax.spines['bottom'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax.xaxis.tick_top()
    ax.tick_params(labeltop=False)  # don't put tick labels at the top
    ax2.xaxis.tick_bottom()

    d = .015  # how big to make the diagonal lines in axes coordinates
    # arguments to pass to plot, just so we don't keep repeating them
    kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
    ax.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
    ax.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

    kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
    ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
    ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal
    plt.savefig(filename)
    plt.show()
    

plt.figure()
plt.hist(train_pBs, 10)
plt.savefig("pB_untrimmed.png")
plt.show()

broken_hist(train_pBs, 10, 0, 20000, 100000, 850000, "pBs.png")

plt.figure()
plt.hist(trimmed_labels, 10)
plt.savefig("trimmed_labels.png")
plt.show()
    
broken_hist(trimmed_back_labels, 10, 0, 8000, 10000, 400000, "Hist_scaled_labels.png")


In [ ]:
train_pBsl_ds = tf.data.Dataset.from_tensor_slices(
    ({c.input_name: train_norm_snapshots}, 
    {c.output_name_1: train_pBs, 
    c.output_name_2: train_norm_snapshots})) \
        .shuffle(pipeline.snapshot_cnt) \
        .batch(c.batch_size)
val_pBsl_ds = tf.data.Dataset.from_tensor_slices(
    ({c.input_name: val_norm_snapshots}, 
    {c.output_name_1: val_pBs, 
    c.output_name_2: val_norm_snapshots})) \
        .shuffle(pipeline.snapshot_cnt) \
        .batch(c.batch_size)
for method in methods_used:
    plot_with_different_settings(
        train_pBsl_ds, val_pBsl_ds, 
        method, 
        "pBsLabels_{}_".format(function_to_str(method)))

In [ ]:
train_pBl_pBbw_ds = tf.data.Dataset.from_tensor_slices(
    ({c.input_name: train_reduced_normed_snapshots}, 
    {c.output_name_1: train_pBs, 
    c.output_name_2: train_reduced_normed_snapshots},
    {c.output_name_1: train_pB_balanced_weights, 
    c.output_name_2: train_hc_balanced_weights})) \
        .shuffle(pipeline.snapshot_cnt) \
        .batch(c.batch_size)
val_pBl_pBbw_ds = tf.data.Dataset.from_tensor_slices(
    ({c.input_name: val_reduced_normed_snapshots}, 
    {c.output_name_1: val_pBs, 
    c.output_name_2: val_reduced_normed_snapshots}, 
    {c.output_name_1: val_pB_balanced_weights, 
    c.output_name_2: val_hc_balanced_weights})) \
        .shuffle(pipeline.snapshot_cnt) \
        .batch(c.batch_size)
for method in methods_used:
    plot_with_different_settings(
        train_pBl_pBbw_ds, val_pBl_pBbw_ds, 
        method, 
        "pBsAsLabels_pBbAsWeights_{}_".format(function_to_str(method)))